In [58]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import OneHotEncoder,LabelEncoder
from sklearn.ensemble import RandomForestClassifier,GradientBoostingClassifier,BaggingClassifier,AdaBoostClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC

In [59]:
df=pd.read_csv("/kaggle/input/health-and-sleep-statistics/Health_Sleep_Statistics.csv")

In [60]:
df

,User ID,Age,Gender,Sleep Quality,Bedtime,Wake-up Time,Daily Steps,Calories Burned,Physical Activity Level,Dietary Habits,Sleep Disorders,Medication Usage
0,1,25,f,8,23:00,06:30,8000,2500,medium,healthy,no,no
1,2,34,m,7,00:30,07:00,5000,2200,low,unhealthy,yes,yes
2,3,29,f,9,22:45,06:45,9000,2700,high,healthy,no,no
3,4,41,m,5,01:00,06:30,4000,2100,low,unhealthy,yes,no
4,5,22,f,8,23:30,07:00,10000,2800,high,medium,no,no
...,...,...,...,...,...,...,...,...,...,...,...,...
95,96,43,m,7,00:45,07:15,6500,2400,medium,medium,no,no
96,97,33,f,8,23:15,06:15,8500,2600,high,medium,no,no
97,98,46,m,4,01:30,07:00,3000,2000,low,unhealthy,yes,yes
98,99,25,f,9,22:15,06:45,9500,2700,high,healthy,no,no


In [61]:
df["Bedtime"]=pd.to_datetime(df["Bedtime"])
df["Hour_bed"]=df["Bedtime"].dt.hour
df["Min_bed"]=df["Bedtime"].dt.minute

/tmp/ipykernel_36/1982637364.py:1: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df["Bedtime"]=pd.to_datetime(df["Bedtime"])


In [62]:
df["Wake-up Time"]=pd.to_datetime(df["Wake-up Time"])
df["Hour_wake"]=df["Wake-up Time"].dt.hour
df["Min_wake"]=df["Wake-up Time"].dt.minute

/tmp/ipykernel_36/3394868467.py:1: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df["Wake-up Time"]=pd.to_datetime(df["Wake-up Time"])


In [63]:
one_hot=OneHotEncoder(sparse_output=False)
one=one_hot.fit_transform(df[["Gender","Physical Activity Level","Dietary Habits","Sleep Disorders"]])
one=pd.DataFrame(one,columns=one_hot.get_feature_names_out())
label_encoder=LabelEncoder()
Y=label_encoder.fit_transform(df["Medication Usage"])
Y_df=pd.DataFrame(Y,columns=["target"])
# Y_df.columns=Y_df.columns.replace("0","target")

In [64]:
concaty=pd.concat([df,one],axis=1).drop(columns=["Gender","Wake-up Time","Physical Activity Level","Dietary Habits","Sleep Disorders","Bedtime","Medication Usage"])


In [65]:
concaty

,User ID,Age,Sleep Quality,Daily Steps,Calories Burned,Hour_bed,Min_bed,Hour_wake,Min_wake,Gender_f,Gender_m,Physical Activity Level_high,Physical Activity Level_low,Physical Activity Level_medium,Dietary Habits_healthy,Dietary Habits_medium,Dietary Habits_unhealthy,Sleep Disorders_no,Sleep Disorders_yes
0,1,25,8,8000,2500,23,0,6,30,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0
1,2,34,7,5000,2200,0,30,7,0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0
2,3,29,9,9000,2700,22,45,6,45,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
3,4,41,5,4000,2100,1,0,6,30,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0
4,5,22,8,10000,2800,23,30,7,0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,96,43,7,6500,2400,0,45,7,15,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0
96,97,33,8,8500,2600,23,15,6,15,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
97,98,46,4,3000,2000,1,30,7,0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0
98,99,25,9,9500,2700,22,15,6,45,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0


In [66]:
X_train,X_test,Y_train,Y_test=train_test_split(concaty,Y_df,test_size=0.2,random_state=42)

In [67]:
random_classif=RandomForestClassifier()
random_classif.fit(X_train,Y_train)
random_classif.score(X_test,Y_test)

/tmp/ipykernel_36/1617334547.py:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  random_classif.fit(X_train,Y_train)


0.8

In [68]:
grad_classif=GradientBoostingClassifier()
grad_classif.fit(X_train,Y_train)
grad_classif.score(X_test,Y_test)

/opt/conda/lib/python3.10/site-packages/sklearn/ensemble/_gb.py:437: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


0.85

In [69]:
ada_classif=AdaBoostClassifier()
ada_classif.fit(X_train,Y_train)
ada_classif.score(X_test,Y_test)

/opt/conda/lib/python3.10/site-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


0.8

In [70]:
bagg_classif=BaggingClassifier()
bagg_classif.fit(X_train,Y_train)
bagg_classif.score(X_test,Y_test)

/opt/conda/lib/python3.10/site-packages/sklearn/ensemble/_bagging.py:802: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


0.9

In [71]:
log_classif=LogisticRegression()
log_classif.fit(X_train,Y_train)
log_classif.score(X_test,Y_test)

/opt/conda/lib/python3.10/site-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/opt/conda/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.8

In [72]:
svm_classif=SVC()
svm_classif.fit(X_train,Y_train)
svm_classif.score(X_test,Y_test)

/opt/conda/lib/python3.10/site-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


0.85

In [73]:
Deci_classif=DecisionTreeClassifier()
Deci_classif.fit(X_train,Y_train)
Deci_classif.score(X_test,Y_test)

0.85